In [1]:
import shutil
import logging
import os
# Utility Function: Delete all files and directories in a specified directory except for one file
def delete_all_except(directory: str, file_to_keep: str):
    """
    Deletes all files and directories in the specified directory except for the specified file.

    :param directory: Path to the directory.
    :param file_to_keep: Filename to preserve.
    """
    if not os.path.exists(directory):
        logging.warning(f"Directory '{directory}' does not exist.")
        return

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename == file_to_keep:
            logging.info(f"Preserving file: {file_path}")
            continue
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
                logging.info(f"Deleted file: {file_path}")
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                logging.info(f"Deleted directory and its contents: {file_path}")
        except Exception as e:
            logging.error(f"Failed to delete {file_path}. Reason: {e}")

current_directory = os.getcwd()
current_directory
file_to_preserve = "google_news_results"
delete_all_except(current_directory, file_to_preserve)

In [2]:
!pip install yfinance pandas numpy matplotlib seaborn gym stable-baselines3 shimmy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
import pandas as pd
from urllib.parse import urlencode
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import time
import os
from urllib.parse import urlparse
import shutil
from transformers import pipeline

def generate_google_news_link(query, start_date, end_date):
    """
    Generate a Google News search URL with custom date range.

    Parameters:
        query (str): The search query.
        start_date (str): Start date in MM/DD/YYYY format.
        end_date (str): End date in MM/DD/YYYY format.

    Returns:
        str: Google News search URL.
    """
    base_url = "https://www.google.com/search"

    params = {
        "q": query,
        "tbs": f"cdr:1,cd_min:{start_date},cd_max:{end_date}",
        "tbm": "nws"
    }

    return f"{base_url}?{urlencode(params)}"

# List of tickers and their homologous terms
search_queries = [
    ["SP 500", "S&P 500", "Standard and Poor's 500", "^GSPC", "S&P 500 Index", "SPX", "S&P 500 ETF", "S&P500", "S&P Index", "Standard & Poor's 500 Index", "S&P 500 Stock Market", "US Stock Market", "American Stocks", "USA Economy", "U.S. Markets", "U.S. Economy", "Wall Street Index", "US Equity Market", "U.S. Stock Exchange", "S&P 500 Companies"],
    ["NASDAQ", "NASDAQ Composite", "NASDAQ Index", "^IXIC", "Nasdaq Composite Index", "NASDAQ-100", "Nasdaq 100", "NASDAQ-100 Index", "NASDAQ stocks", "NASDAQ Index ETF", "American Technology Stocks", "U.S. Tech Stocks", "Tech-heavy Index", "USA Stock Market", "Nasdaq Tech Index", "NASDAQ Growth", "Silicon Valley Stocks", "NASDAQ 100 Tech", "USA Technology", "Tech Stocks Index"],
    ["Dow Jones", "DJIA", "Dow Jones Industrial Average", "^DJI", "Dow Jones Index", "Dow Jones Average", "DJIA Index", "Dow Jones Industrial", "Dow Jones Industrial Stocks", "DJIA ETF", "US Blue-Chip Stocks", "USA Industrial Stocks", "U.S. Market Leaders", "Wall Street Benchmark", "Dow Jones Companies", "US Industrials", "American Economy", "US Industrial Market", "US Stock Index", "Wall Street Giants"],
    ["CAC 40", "Paris Stock Exchange", "Euronext Paris", "^FCHI", "French Stock Market", "Paris Index", "French Economy", "France Stock Exchange", "CAC 40 Companies", "Paris Bourse", "French Markets", "Paris Exchange", "Euronext Index", "French Blue Chip Stocks", "French Equity Market", "France Stock Index", "Paris Market", "Eurozone Stocks", "France Economy", "Eurozone Market"],
    ["FTSE 100", "London Stock Exchange", "UK 100 Index", "^FTSE", "FTSE 100 Index", "London Index", "UK Stock Market", "British Stock Exchange", "UK Economy", "FTSE 100 Companies", "London Market", "UK Economy Stocks", "FTSE Index ETF", "British Blue Chips", "London Exchange", "UK Markets", "British Economy", "UK Financial Markets", "London Stock Index", "UK Stock Exchange", "FTSE 100 Stocks"],
    ["^STOXX50E", "EuroStoxx 50", "EuroStoxx 50 Index", "European Stock Market", "Eurozone Stocks", "European Economy", "Stoxx Europe 50", "Eurozone 50 Index", "EuroStoxx Index", "Europe Market Leaders", "Eurozone Leaders", "Top European Stocks", "Eurozone Top Companies", "European Blue Chips", "European Equity Market", "Eurozone Financials", "Eurozone Benchmark", "European Blue Chip Stocks", "Eurozone Economic Index", "European Market Index"],
    ["^N225", "Nikkei 225", "Nikkei Index", "Japanese Stock Market", "Japan Economy", "Nikkei Average", "Tokyo Stock Exchange", "Japan Top 225 Stocks", "Japan Stock Index", "Japanese Equity Market", "Nikkei 225 Companies", "Japan's Leading Stocks", "Japanese Financial Market", "Tokyo Exchange", "Nikkei Market", "Japan Economic Index", "Japanese Economy", "Japan's Stock Exchange", "Top Japanese Companies", "Nikkei 225 ETF"],
    ["^HSI", "Hang Seng", "Hang Seng Index", "Hong Kong Stock Market", "Hong Kong Economy", "Hong Kong Exchange", "HSI Index", "Hang Seng Index ETF", "Hong Kong Financial Market", "Chinese Stock Market", "HSI Stocks", "Asia-Pacific Stocks", "Hong Kong Blue Chips", "Hong Kong's Leading Stocks", "HSI Index Stocks", "Asian Financial Market", "Hong Kong Leading Companies", "Asian Market Leaders", "Hang Seng Companies", "Hong Kong Stock Index"],
    ["000001.SS", "Shanghai Composite", "Shanghai Stock Exchange", "Chinese Stock Market", "China Economy", "Shanghai Index", "China Financial Market", "Shanghai Composite Index", "Shanghai Exchange", "Chinese Stock Index", "China's Leading Stocks", "Shanghai Exchange Companies", "China Blue Chip Stocks", "Chinese Equity Market", "Shanghai Financial Index", "China Benchmark", "Shanghai Composite ETF", "China's Leading Companies", "Chinese Market Leaders", "China's Economic Stocks"],
    ["^BSESN", "Bombay Sensex", "S&P BSE Sensex", "Indian Stock Market", "India Economy", "BSE Sensex", "Mumbai Stock Exchange", "Sensex Companies", "Indian Financial Market", "Sensex 30", "BSE 30 Index", "India's Leading Stocks", "Indian Market Leaders", "Indian Equity Market", "Bombay Exchange", "Indian Stock Index", "Sensex Index ETF", "BSE India", "Indian Economy Stocks", "Bombay Financial Index"],
    ["^NSEI", "Nifty 50", "National Stock Exchange of India", "Indian Stock Index", "India Stock Market", "Nifty Index", "India Economy", "Indian Market Leaders", "Nifty 50 Stocks", "NSE India", "Indian Blue Chips", "Indian Financial Market", "Indian Stock Market", "India 50 Index", "Nifty Index ETF", "India's Top 50", "India's Leading Stocks", "Nifty 50 Companies", "India Economic Stocks", "Indian Market Index"],
    ["^KS11", "KOSPI", "Korea Composite Stock Price Index", "Korean Stock Market", "Korea Economy", "KOSPI Index", "South Korean Stock Market", "Korean Market Leaders", "KOSPI 200", "South Korea Financial Market", "Korean Exchange", "KOSPI ETF", "Korean Leading Stocks", "South Korean Economy", "Korean Market Index", "South Korea Stock Index", "Korean Economy Stocks", "South Korea Exchange", "Korean Equity Market", "Korea Stock Index"],
    ["Gold", "XAU", "Gold Price", "Gold Market", "Precious Metals", "Gold Spot", "Gold Bullion", "Gold ETF", "Gold Investment", "Gold Mining", "Gold Futures", "Gold Stocks", "Gold Index", "Gold Commodity", "Gold Trading", "Gold Bullion ETF", "Gold Commodity Index", "Gold Market Trends", "Gold Investment Funds", "Gold Prices Today"],
    ["Silver", "XAG", "Silver Price", "Silver Market", "Precious Metals", "Silver Spot", "Silver Bullion", "Silver ETF", "Silver Investment", "Silver Mining", "Silver Futures", "Silver Stocks", "Silver Index", "Silver Commodity", "Silver Trading", "Silver Bullion ETF", "Silver Commodity Index", "Silver Market Trends", "Silver Investment Funds", "Silver Prices Today"],
    ["Oil", "Crude Oil", "WTI", "Brent Crude", "Oil Price", "Crude Oil Price", "OPEC", "Oil Futures", "Oil Market", "Oil Stocks", "Oil ETF", "Global Oil Supply", "Oil Trading", "Oil Production", "Brent Oil Futures", "Oil Price Index", "Oil Investment", "Oil Exploration", "Oil Trading Market", "Oil Price Trends"]
]


# Define the year range
start_year = 2003
end_year = 2024

# Store all generated links
links = []

for query_group in search_queries:  # Loop through each group of related terms
    for query in query_group:  # Loop through each term in the group
        for year in range(start_year, end_year + 1):  # Loop through years
            for month in range(1, 13):  # Loop through months
                start_date = f"{month}/1/{year}"
                # Calculate the last day of the month
                next_month = month % 12 + 1
                next_year = year if month < 12 else year + 1
                end_date = (datetime(next_year, next_month, 1) - timedelta(days=1)).strftime("%m/%d/%Y")

                # Generate the search link
                link = generate_google_news_link(query, start_date, end_date)
                links.append([query, start_date, end_date, link])

# Convert to DataFrame
df = pd.DataFrame(links, columns=["Query", "Start Date", "End Date", "Google News Link"])

# Save to CSV
csv_filename = "links.csv"
df.to_csv(csv_filename, index=False)

# Create the NLP_data folder if it doesn't exist
os.makedirs("NLP_data", exist_ok=True)

# Move the links.csv file into the NLP_data folder
shutil.move("links.csv", os.path.join("NLP_data", "links.csv"))


'NLP_data/links.csv'

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os
from urllib.parse import urlencode
from datetime import datetime, timedelta

def generate_google_news_link(query, start_date, end_date):
    """
    Generate a Google News search URL with custom date range.
    """
    base_url = "https://www.google.com/search"
    params = {
        "q": query,
        "tbs": f"cdr:1,cd_min:{start_date},cd_max:{end_date}",
        "tbm": "nws"
    }
    return f"{base_url}?{urlencode(params)}"

def get_news_links(search_url):
    """
    Fetch news article links from a Google News search results page.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(search_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract all links from the search results
        links = []
        for a_tag in soup.find_all("a", href=True):
            href = a_tag["href"]
            if "https://" in href and "google.com" not in href:
                links.append(href)
        return set(links)  # Remove duplicates
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve {search_url}: {e}")
        return []

# Load CSV file with search queries from the NLP_data folder
csv_file_path = os.path.join("NLP_data", "links.csv")
df = pd.read_csv(csv_file_path)

# Base folder to save results
output_base_folder = "google_news_results"
os.makedirs(output_base_folder, exist_ok=True)

# Process each row in the CSV
for index, row in df.iterrows():
    query = row["Query"].replace(" ", "_")  # Sanitize folder name
    search_url = row["Google News Link"]

    # Create directory for the query
    query_folder = os.path.join(output_base_folder, query)
    os.makedirs(query_folder, exist_ok=True)

    # Get news article links
    news_links = get_news_links(search_url)

    # Sanitize file name by replacing slashes with underscores
    start_date_sanitized = row['Start Date'].replace("/", "_")
    end_date_sanitized = row['End Date'].replace("/", "_")
    file_path = os.path.join(query_folder, f"{start_date_sanitized}_to_{end_date_sanitized}.txt")

    # Save links to a text file
    with open(file_path, "w", encoding="utf-8") as f:
        for link in news_links:
            f.write(link + "\n")

    print(f"Saved {len(news_links)} links for {query} ({row['Start Date']} to {row['End Date']})")

    # Respect Google's policies, wait between requests
    time.sleep(3)


Saved 0 links for SP_500 (1/1/2003 to 01/31/2003)
Saved 2 links for SP_500 (2/1/2003 to 02/28/2003)


KeyboardInterrupt: 

In [ ]:
import os
from datetime import datetime
import shutil

# Base folder where raw results are stored
output_base_folder = "google_news_results"

# Move non-empty files and delete empty ones
for query in os.listdir(output_base_folder):
    query_path = os.path.join(output_base_folder, query)
    if os.path.isdir(query_path):
        for file_name in os.listdir(query_path):
            file_path = os.path.join(query_path, file_name)
            if os.path.isfile(file_path):
                # Delete file if it is empty
                if os.path.getsize(file_path) == 0:
                    os.remove(file_path)
                    print(f"Deleted empty file: {file_path}")
                else:
                    # Extract the date part from the filename (assumes format: MM_DD_YYYY_to_...)
                    try:
                        date_part = file_name.split("_to_")[0]  # Get the start date part
                        date_obj = datetime.strptime(date_part, "%m_%d_%Y")  # Convert to datetime
                        year = date_obj.year
                        month = date_obj.month
                        month_name = date_obj.strftime("%B")  # Full month name

                        # New folder structure: output_base_folder/year/MonthName/query
                        new_location = os.path.join(output_base_folder, str(year), month_name, query)
                        os.makedirs(new_location, exist_ok=True)

                        # Move the file to the new location
                        os.rename(file_path, os.path.join(new_location, file_name))
                        print(f"Moved file to {new_location}")
                    except ValueError:
                        print(f"Skipping file due to incorrect date format: {file_name}")

# Delete any now-empty query folders at the top level of output_base_folder
for query in os.listdir(output_base_folder):
    query_path = os.path.join(output_base_folder, query)
    if os.path.isdir(query_path) and not os.listdir(query_path):
        os.rmdir(query_path)
        print(f"Deleted empty group folder: {query_path}")

print("Cleanup and organization complete!")


In [ ]:
import os

# Dictionary mapping standardized query groups to their variants
query_groups = {
    "SP_500": ["SP_500", "S&P_500", "Standard_and_Poor's_500", "^GSPC", "S&P_500_Index", "SPX", "S&P_500_ETF", "S&P500", "S&P_Index", "Standard_&_Poor's_500_Index", "S&P_500_Stock_Market", "US_Stock_Market", "American_Stocks", "USA_Economy", "U.S._Markets", "U.S._Economy", "Wall_Street_Index", "US_Equity_Market", "U.S._Stock_Exchange", "S&P_500_Companies"],
    "NASDAQ": ["NASDAQ", "NASDAQ_Composite", "NASDAQ_Index", "^IXIC", "Nasdaq_Composite_Index", "NASDAQ-100", "Nasdaq_100", "NASDAQ-100_Index", "NASDAQ_stocks", "NASDAQ_Index_ETF", "American_Technology_Stocks", "U.S._Tech_Stocks", "Tech-heavy_Index", "USA_Stock_Market", "Nasdaq_Tech_Index", "NASDAQ_Growth", "Silicon_Valley_Stocks", "NASDAQ_100_Tech", "USA_Technology", "Tech_Stocks_Index"],
    "Dow_Jones": ["Dow_Jones", "DJIA", "Dow_Jones_Industrial_Average", "^DJI", "Dow_Jones_Index", "Dow_Jones_Average", "DJIA_Index", "Dow_Jones_Industrial", "Dow_Jones_Industrial_Stocks", "DJIA_ETF", "US_Blue-Chip_Stocks", "USA_Industrial_Stocks", "U.S._Market_Leaders", "Wall_Street_Benchmark", "Dow_Jones_Companies", "US_Industrials", "American_Economy", "US_Industrial_Market", "US_Stock_Index", "Wall_Street_Giants"],
    "CAC_40": ["CAC_40", "Paris_Stock_Exchange", "Euronext_Paris", "^FCHI", "French_Stock_Market", "Paris_Index", "French_Economy", "France_Stock_Exchange", "CAC_40_Companies", "Paris_Bourse", "French_Markets", "Paris_Exchange", "Euronext_Index", "French_Blue_Chip_Stocks", "French_Equity_Market", "France_Stock_Index", "Paris_Market", "Eurozone_Stocks", "France_Economy", "Eurozone_Market"],
    "FTSE_100": ["FTSE_100", "London_Stock_Exchange", "UK_100_Index", "^FTSE", "FTSE_100_Index", "London_Index", "UK_Stock_Market", "British_Stock_Exchange", "UK_Economy", "FTSE_100_Companies", "London_Market", "UK_Economy_Stocks", "FTSE_Index_ETF", "British_Blue_Chips", "London_Exchange", "UK_Markets", "British_Economy", "UK_Financial_Markets", "London_Stock_Index", "UK_Stock_Exchange", "FTSE_100_Stocks"],
    "EuroStoxx_50": ["^STOXX50E", "EuroStoxx_50", "EuroStoxx_50_Index", "European_Stock_Market", "Eurozone_Stocks", "European_Economy", "Stoxx_Europe_50", "Eurozone_50_Index", "EuroStoxx_Index", "Europe_Market_Leaders", "Eurozone_Leaders", "Top_European_Stocks", "Eurozone_Top_Companies", "European_Blue_Chips", "European_Equity_Market", "Eurozone_Financials", "Eurozone_Benchmark", "European_Blue_Chip_Stocks", "Eurozone_Economic_Index", "European_Market_Index"],
    "Nikkei_225": ["^N225", "Nikkei_225", "Nikkei_Index", "Japanese_Stock_Market", "Japan_Economy", "Nikkei_Average", "Tokyo_Stock_Exchange", "Japan_Top_225_Stocks", "Japan_Stock_Index", "Japanese_Equity_Market", "Nikkei_225_Companies", "Japan's_Leading_Stocks", "Japanese_Financial_Market", "Tokyo_Exchange", "Nikkei_Market", "Japan_Economic_Index", "Japanese_Economy", "Japan's_Stock_Exchange", "Top_Japanese_Companies", "Nikkei_225_ETF"],
    "Hang_Seng": ["^HSI", "Hang_Seng", "Hang_Seng_Index", "Hong_Kong_Stock_Market", "Hong_Kong_Economy", "Hong_Kong_Exchange", "HSI_Index", "Hang_Seng_Index_ETF", "Hong_Kong_Financial_Market", "Chinese_Stock_Market", "HSI_Stocks", "Asia-Pacific_Stocks", "Hong_Kong_Blue_Chips", "Hong_Kong's_Leading_Stocks", "HSI_Index_Stocks", "Asian_Financial_Market", "Hong_Kong_Leading_Companies", "Asian_Market_Leaders", "Hang_Seng_Companies", "Hong_Kong_Stock_Index"],
    "Shanghai_Composite": ["000001.SS", "Shanghai_Composite", "Shanghai_Stock_Exchange", "Chinese_Stock_Market", "China_Economy", "Shanghai_Index", "China_Financial_Market", "Shanghai_Composite_Index", "Shanghai_Exchange", "Chinese_Stock_Index", "China's_Leading_Stocks", "Shanghai_Exchange_Companies", "China_Blue_Chip_Stocks", "Chinese_Equity_Market", "Shanghai_Financial_Index", "China_Benchmark", "Shanghai_Composite_ETF", "China's_Leading_Companies", "Chinese_Market_Leaders", "China's_Economic_Stocks"],
    "Bombay_Sensex": ["^BSESN", "Bombay_Sensex", "S&P_BSE_Sensex", "Indian_Stock_Market", "India_Economy", "BSE_Sensex", "Mumbai_Stock_Exchange", "Sensex_Companies", "Indian_Financial_Market", "Sensex_30", "BSE_30_Index", "India's_Leading_Stocks", "Indian_Market_Leaders", "Indian_Equity_Market", "Bombay_Exchange", "Indian_Stock_Index", "Sensex_Index_ETF", "BSE_India", "Indian_Economy_Stocks", "Bombay_Financial_Index"],
    "Nifty_50": ["^NSEI", "Nifty_50", "National_Stock_Exchange_of_India", "Indian_Stock_Index", "India_Stock_Market", "Nifty_Index", "India_Economy", "Indian_Market_Leaders", "Nifty_50_Stocks", "NSE_India", "Indian_Blue_Chips", "Indian_Financial_Market", "Indian_Stock_Market", "India_50_Index", "Nifty_Index_ETF", "India's_Top_50", "India's_Leading_Stocks", "Nifty_50_Companies", "India_Economic_Stocks", "Indian_Market_Index"],
    "KOSPI": ["^KS11", "KOSPI", "Korea_Composite_Stock_Price_Index", "Korean_Stock_Market", "Korea_Economy", "KOSPI_Index", "South_Korean_Stock_Market", "Korean_Market_Leaders", "KOSPI_200", "South_Korea_Financial_Market", "Korean_Exchange", "KOSPI_ETF", "Korean_Leading_Stocks", "South_Korean_Economy", "Korean_Market_Index", "South_Korea_Stock_Index", "Korean_Economy_Stocks", "South_Korea_Exchange", "Korean_Equity_Market", "Korea_Stock_Index"],
    "Gold": ["Gold", "XAU", "Gold_Price", "Gold_Market", "Precious_Metals", "Gold_Spot", "Gold_Bullion", "Gold_ETF", "Gold_Investment", "Gold_Mining", "Gold_Futures", "Gold_Stocks", "Gold_Index", "Gold_Commodity", "Gold_Trading", "Gold_Bullion_ETF", "Gold_Commodity_Index", "Gold_Market_Trends", "Gold_Investment_Funds", "Gold_Prices_Today"],
    "Silver": ["Silver", "XAG", "Silver_Price", "Silver_Market", "Precious_Metals", "Silver_Spot", "Silver_Bullion", "Silver_ETF", "Silver_Investment", "Silver_Mining", "Silver_Futures", "Silver_Stocks", "Silver_Index", "Silver_Commodity", "Silver_Trading", "Silver_Bullion_ETF", "Silver_Commodity_Index", "Silver_Market_Trends", "Silver_Investment_Funds", "Silver_Prices_Today"],
    "Oil": ["Oil", "Crude_Oil", "WTI", "Brent_Crude", "Oil_Price", "Crude_Oil_Price", "OPEC", "Oil_Futures", "Oil_Market", "Oil_Stocks", "Oil_ETF", "Global_Oil_Supply", "Oil_Trading", "Oil_Production", "Brent_Oil_Futures", "Oil_Price_Index", "Oil_Investment", "Oil_Exploration", "Oil_Trading_Market", "Oil_Price_Trends"]
}

# Loop through each year/month folder and unify text files for each query group
for year_folder in os.listdir(output_base_folder):
    year_path = os.path.join(output_base_folder, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                for query_group, query_list in query_groups.items():
                    unified_file_path = os.path.join(month_path, f"{query_group}_united.txt")
                    with open(unified_file_path, "w", encoding="utf-8") as unified_file:
                        for query in query_list:
                            query_folder_path = os.path.join(month_path, query)
                            if os.path.isdir(query_folder_path):
                                for file_name in os.listdir(query_folder_path):
                                    file_path = os.path.join(query_folder_path, file_name)
                                    if file_path.endswith(".txt") and os.path.isfile(file_path):
                                        with open(file_path, "r", encoding="utf-8") as f:
                                            content = f.read()
                                            unified_file.write(content + "\n")
                                        os.remove(file_path)
                                        print(f"Deleted {file_path}")
                                # Remove the now-empty query folder
                                os.rmdir(query_folder_path)
                                print(f"Deleted empty folder: {query_folder_path}")
                    print(f"Created unified file: {unified_file_path}")


In [ ]:
import os

def remove_empty_lines(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    non_empty_lines = [line for line in lines if line.strip() != ""]
    with open(file_path, "w", encoding="utf-8") as f:
        f.writelines(non_empty_lines)
    print(f"Removed empty lines from: {file_path}")

# Loop through each year/month folder and clean text files
for year_folder in os.listdir(output_base_folder):
    year_path = os.path.join(output_base_folder, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                for query_folder in os.listdir(month_path):
                    query_folder_path = os.path.join(month_path, query_folder)
                    if os.path.isdir(query_folder_path):
                        for file_name in os.listdir(query_folder_path):
                            file_path = os.path.join(query_folder_path, file_name)
                            if file_path.endswith(".txt") and os.path.isfile(file_path):
                                remove_empty_lines(file_path)
print("Empty lines removal completed.")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_article_details(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else "No title found"
        paragraphs = soup.find_all('p')
        article_text = " ".join([p.get_text() for p in paragraphs])
        first_500_words = " ".join(article_text.split()[:500])
        return title, first_500_words
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")
        return "Error", "Error"

# Process unified text files to extract article details and save as CSV
for year_folder in os.listdir(output_base_folder):
    year_path = os.path.join(output_base_folder, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                for file_name in os.listdir(month_path):
                    if file_name.endswith("_united.txt"):
                        file_path = os.path.join(month_path, file_name)
                        with open(file_path, "r", encoding="utf-8") as f:
                            urls = f.readlines()
                        article_data = []
                        for url in urls:
                            url = url.strip()
                            title, first_500_words = extract_article_details(url)
                            article_data.append([url, title, first_500_words])
                        csv_file_path = os.path.splitext(file_path)[0] + ".csv"
                        df = pd.DataFrame(article_data, columns=["URL", "Title", "First_500_Words"])
                        df.to_csv(csv_file_path, index=False, encoding="utf-8")
                        print(f"Created CSV for {file_name}: {csv_file_path}")
print("Data extraction and CSV creation completed.")


In [ ]:
import shutil
import os

# Define source and target base folders
source_base_folder = "google_news_results"
target_base_folder = "sentiment_analysis"
os.makedirs(target_base_folder, exist_ok=True)

# Move CSV files from source to target structure, preserving year/month hierarchy
for year_folder in os.listdir(source_base_folder):
    year_path = os.path.join(source_base_folder, year_folder)
    if os.path.isdir(year_path):
        target_year_path = os.path.join(target_base_folder, year_folder)
        os.makedirs(target_year_path, exist_ok=True)
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                target_month_path = os.path.join(target_year_path, month_folder)
                os.makedirs(target_month_path, exist_ok=True)
                for file_name in os.listdir(month_path):
                    if file_name.endswith(".csv"):
                        file_path = os.path.join(month_path, file_name)
                        target_file_path = os.path.join(target_month_path, file_name)
                        shutil.move(file_path, target_file_path)
                        print(f"Moved {file_name} to {target_file_path}")
print("CSV files have been moved to the sentiment_analysis folder.")


In [ ]:
import pandas as pd
from transformers import pipeline
import os

# Load FinBERT sentiment analysis model
sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert")
def add_sentiment_labels(csv_path):
    df = pd.read_csv(csv_path)
    if 'First_500_Words' not in df.columns:
        print(f"Skipping {csv_path} (no 'First_500_Words' column found)")
        return
    sentiment_labels = []
    sentiment_scores = []
    for text in df['First_500_Words']:
        if isinstance(text, str):
            truncated_text = text[:500]  # Truncate text if needed
            sentiment = sentiment_pipeline(truncated_text)[0]
            sentiment_labels.append(sentiment['label'])
            sentiment_scores.append(sentiment['score'])
        else:
            sentiment_labels.append("Error")
            sentiment_scores.append(0.0)
    df['Sentiment_Label'] = sentiment_labels
    df['Sentiment_Score'] = sentiment_scores
    df.to_csv(csv_path, index=False, encoding="utf-8")
    print(f"Updated sentiment for {csv_path}")

# Apply sentiment analysis to each CSV in sentiment_analysis folder
sentiment_base_folder = "sentiment_analysis"
for year_folder in os.listdir(sentiment_base_folder):
    year_path = os.path.join(sentiment_base_folder, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                for file_name in os.listdir(month_path):
                    if file_name.endswith(".csv"):
                        csv_path = os.path.join(month_path, file_name)
                        add_sentiment_labels(csv_path)
print("Sentiment analysis labels and scores have been added.")


In [ ]:
import pandas as pd
import os
def calculate_average_score(csv_path):
    df = pd.read_csv(csv_path)
    if 'Sentiment_Label' not in df.columns or 'Sentiment_Score' not in df.columns:
        print(f"Skipping {csv_path} (required columns missing)")
        return None
    positive_scores = df[df['Sentiment_Label'] == 'positive']['Sentiment_Score']
    negative_scores = df[df['Sentiment_Label'] == 'negative']['Sentiment_Score']
    sum_positive = positive_scores.sum()
    sum_negative = negative_scores.sum()
    num_positive = len(positive_scores)
    num_negative = len(negative_scores)
    if num_positive + num_negative == 0:
        return None
    average_score = (sum_positive - sum_negative) / (num_positive + num_negative)
    return average_score
# Process each month folder to create a summary CSV
sentiment_base_folder = "sentiment_analysis"
for year_folder in os.listdir(sentiment_base_folder):
    year_path = os.path.join(sentiment_base_folder, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                summary_data = []
                print(f"Processing {month_folder}...")
                for file_name in os.listdir(month_path):
                    if file_name.endswith(".csv"):
                        csv_path = os.path.join(month_path, file_name)
                        # Extract the query group name from filename (remove _united.csv)
                        query_group_name = file_name.replace("_united.csv", "")
                        avg_score = calculate_average_score(csv_path)
                        if avg_score is not None:
                            print(f"Adding {query_group_name} with score: {avg_score}")
                            summary_data.append([query_group_name, avg_score])
                if summary_data:
                    summary_df = pd.DataFrame(summary_data, columns=["Query Group", "Average Score"])
                    summary_csv_path = os.path.join(month_path, "monthly_summary.csv")
                    summary_df.to_csv(summary_csv_path, index=False, encoding="utf-8")
                    print(f"Created monthly summary for {month_folder}: {summary_csv_path}")
                else:
                    print(f"No data for {month_folder}, skipping.")
print("Monthly summaries have been created.")


In [ ]:
import pandas as pd
import os
import shutil

# Define base folders for final results
sentiment_base_folder = "sentiment_analysis"
final_results_folder = "final_results"
os.makedirs(final_results_folder, exist_ok=True)

# Process monthly summaries and copy them to final_results, while aggregating data for unified matrix
all_months_data = []

for year_folder in os.listdir(sentiment_base_folder):
    year_path = os.path.join(sentiment_base_folder, year_folder)
    if os.path.isdir(year_path):
        target_year_path = os.path.join(final_results_folder, year_folder)
        os.makedirs(target_year_path, exist_ok=True)
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                summary_csv_path = os.path.join(month_path, "monthly_summary.csv")
                if os.path.exists(summary_csv_path):
                    target_month_path = os.path.join(target_year_path, month_folder)
                    os.makedirs(target_month_path, exist_ok=True)
                    shutil.copy(summary_csv_path, target_month_path)
                    month_data = pd.read_csv(summary_csv_path)
                    month_data['Month'] = month_folder
                    month_data['Year'] = year_folder
                    all_months_data.append(month_data)

# Create unified matrix if data is available
if all_months_data:
    unified_df = pd.concat(all_months_data)
    unified_matrix = unified_df.pivot_table(index=['Year', 'Month'], columns='Query Group', values='Average Score')
    unified_matrix_path = os.path.join(final_results_folder, "unified_matrix.csv")
    unified_matrix.to_csv(unified_matrix_path, encoding="utf-8")
    print(f"Unified matrix has been created at: {unified_matrix_path}")
else:
    print("No data available to create the unified matrix.")

# (Optional) Save a simulated unified matrix as a separate file
unified_matrix.to_csv("simulated_unified_matrix.csv", encoding="utf-8")
print("Simulated unified matrix saved to simulated_unified_matrix.csv")
